In [1]:
! git clone https://github.com/ramybaly/Article-Bias-Prediction

fatal: destination path 'Article-Bias-Prediction' already exists and is not an empty directory.


In [2]:
import pandas as pd
import os
import json
from tqdm import tqdm

In [3]:
raw_ds_path = './Article-Bias-Prediction/data/jsons'
json_files = os.listdir(raw_ds_path)

In [40]:
df = pd.DataFrame()
for file_path in tqdm(json_files):
  with open(raw_ds_path + '/' + file_path) as f:
    n = json.load(f)
  i = n['ID']
  for feature in [f for f in n.keys() if f not in ['ID']]:
    df.loc[i, feature] = n[feature]

df.head()
df.index.name = 'id'

100%|██████████| 37554/37554 [08:19<00:00, 75.24it/s] 


In [44]:
pd.to_datetime(df['date'], format='%m/%d/%y', errors="coerce").describe()

count                               37
mean     2018-01-12 22:03:14.594594560
min                2016-07-11 00:00:00
25%                2017-04-04 00:00:00
50%                2018-01-21 00:00:00
75%                2018-09-05 00:00:00
max                2019-08-19 00:00:00
Name: date, dtype: object

In [45]:
pd.to_datetime(df['date'], format='%Y-%m-%d', errors="coerce").describe()

count                            33108
mean     2016-10-29 04:09:49.778905088
min                2001-10-23 00:00:00
25%                2014-09-17 00:00:00
50%                2017-01-10 00:00:00
75%                2019-01-25 00:00:00
max                2050-09-30 00:00:00
Name: date, dtype: object

In [48]:
df.to_csv('./dataset/polarization_ds.csv')

In [49]:
train_media_split = pd.read_csv('./Article-Bias-Prediction/data/splits/media/train.tsv', sep='\t')

valid_media_split = pd.read_csv('./Article-Bias-Prediction/data/splits/media/valid.tsv', sep='\t')

test_media_split = pd.read_csv('./Article-Bias-Prediction/data/splits/media/test.tsv', sep='\t')

In [52]:
train_df = df.loc[train_media_split['ID']]
valid_df = df.loc[valid_media_split['ID']]
test_df = df.loc[test_media_split['ID']]

In [55]:
train_df['topic']

id
zl7kc7EmAyIdUMIo    immigration
xpbjYTJYPdlw6HmJ        culture
k4SGI3GXarnz5dJl      elections
0jIpietfnrPRGHKQ    white_house
zMlSt7dyJvanHqJq       politics
                       ...     
GXYz0Td0RuvzCPbk    immigration
3evIA5MtIVCSzmYK    middle_east
E0IUo5gfHaEgzMOb    white_house
40WIsjRfR0lCKcjf    coronavirus
QefeiRX9myBBP9HH       politics
Name: topic, Length: 26590, dtype: object

In [82]:
from collections import Counter

train_t = Counter(train_df['topic'])
valid_t = Counter(valid_df['topic'])
test_t = Counter(test_df['topic'])

In [89]:
train_freq = {k: c / len(train_df) for k, c in train_t.items()}
valid_freq = {k: c / len(valid_df) for k, c in valid_t.items()}
test_freq = {k: c / len(test_df) for k, c in test_t.items()}

In [108]:
import numpy as np
def diff(f1, f2):
    return [np.abs(f1[k] - f2.get(k, 0)) for k in f1.keys()]
tr_te_diff = diff(train_freq, test_freq)

In [113]:
sorted({k: np.abs(train_freq[k] - test_freq.get(k, 0)) for k in train_freq.keys()}.items(), key = lambda it: -it[1])

[('politics', 0.4091804322041253),
 ('elections', 0.1557352388115833),
 ('media_bias', 0.06921254375560507),
 ('race_and_racism', 0.06690890155350478),
 ('white_house', 0.05024445280180519),
 ('lgbt_rights', 0.04671420719183036),
 ('fbi', 0.043616744293690515),
 ('immigration', 0.04125611132004513),
 ('healthcare', 0.0397141782625047),
 ('middle_east', 0.025723956374576907),
 ('coronavirus', 0.02523505077096653),
 ('national_defense', 0.02269042728613996),
 ('federal_budget', 0.020238666936673703),
 ('us_senate', 0.020195562241444153),
 ('us_house', 0.01831515607371192),
 ('economy_and_jobs', 0.017863858593456187),
 ('violence_in_america', 0.01763820985332832),
 ('world', 0.015419330575404287),
 ('cybersecurity', 0.015039199236265805),
 ('us_congress', 0.014855208725084619),
 ('nuclear_weapons', 0.014799953713078948),
 ('gun_control_and_gun_rights', 0.014763213469494026),
 ('foreign_policy', 0.01421587062805566),
 ('national_security', 0.013388491914253479),
 ('terrorism', 0.0132380594

In [119]:
train_freq['politics'], test_freq['politics'], train_freq['elections']

(0.07235802933433622, 0.4815384615384615, 0.1557352388115833)

In [97]:
train_topics = set(train_freq.keys())
valid_topics = set(valid_freq.keys())
test_topics = set(test_freq.keys())

In [104]:
len(valid_topics - train_topics), len(train_topics - valid_topics), len(train_topics - test_topics), len(test_topics - train_topics)

(0, 6, 81, 0)

In [92]:
valid_freq - train_freq

TypeError: unsupported operand type(s) for -: 'dict' and 'dict'

In [77]:
test_t

Counter({'media_bias': 133,
         'culture': 5,
         'federal_budget': 51,
         'opioid_crisis': 4,
         'politics': 626,
         'education': 29,
         'race_and_racism': 106,
         'environment': 8,
         'lgbt_rights': 73,
         'fbi': 70,
         'gun_control_and_gun_rights': 12,
         'national_defense': 36,
         'cybersecurity': 24,
         'supreme_court': 21,
         'labor': 10,
         'justice_department': 14,
         'us_military': 6,
         'nuclear_weapons': 21,
         'republican_party': 13,
         'palestine': 7,
         'polarization': 4,
         'banking_and_finance': 8,
         'great_britain': 6,
         'justice': 4,
         'trade': 1,
         'treasury': 6,
         'mexico': 2})

In [74]:
train_t - test_t, valid_t - test_t

(Counter({'immigration': 1097,
          'culture': 207,
          'elections': 4141,
          'white_house': 1336,
          'politics': 1298,
          'north_korea': 206,
          'impeachment': 278,
          'middle_east': 684,
          'environment': 415,
          'supreme_court': 548,
          'nsa': 183,
          'lgbt_rights': 178,
          'free_speech': 166,
          'media_bias': 747,
          'world': 410,
          'transportation': 43,
          'ebola': 68,
          'sports': 63,
          'republican_party': 365,
          'justice': 205,
          'gun_control_and_gun_rights': 626,
          'us_military': 170,
          'democratic_party': 152,
          'coronavirus': 671,
          'economic_policy': 121,
          'fbi': 202,
          'business': 54,
          'cybersecurity': 67,
          'foreign_policy': 378,
          'russia': 125,
          'race_and_racism': 283,
          'taxes': 249,
          'polarization': 304,
          'national_security